In [ ]:
#IMPORT REQUIRED LIBRARIEs
from time import time
import numpy as np
import pandas as pd
import os
from re import search
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Activation,Flatten, Conv2D, MaxPooling2D,Dropout
from  tensorflow.keras.applications import InceptionV3

In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

# Data preparation

In [ ]:
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255
)
INPUT_SIZE = (124,124,3)
test_generator =  test_datagen.flow_from_dataframe(
    submission,
    directory="../input/plant-pathology-2021-fgvc8/test_images",
    x_col='image',
    y_col=None,
    class_mode=None,
    target_size=INPUT_SIZE[:2]
) 

# Model construction

In [ ]:
#initialise base model
IMG_SHAPE = (124, 124, 3)

base_model=  InceptionV3(
    weights='../input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
    input_shape= IMG_SHAPE,
    include_top=False)

In [ ]:
base_model.trainable = True

fine_tune_at = 200

# Freeze all the layers before the 'fine_tune_at' layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:

model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(320, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(12, activation='sigmoid'))
model.summary()

In [ ]:
# Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001/10),
    loss='categorical_crossentropy',
    metrics = [tfa.metrics.F1Score(num_classes = 12,average = "macro",name = "f1_score")])

# Import a model

In [ ]:
model.load_weights("../input/best-model/Best_model_inceptionV3 (1).h5")

# Predictions

In [ ]:
preds = model.predict(test_generator)
print(preds)

In [ ]:
Classes={'complex': 0,
 'frog_eye_leaf_spot': 1,
 'frog_eye_leaf_spot complex': 2,
 'healthy': 3,
 'powdery_mildew': 4,
 'powdery_mildew complex': 5,
 'rust': 6,
 'rust complex': 7,
 'rust frog_eye_leaf_spot': 8,
 'scab': 9,
 'scab frog_eye_leaf_spot': 10,
 'scab frog_eye_leaf_spot complex': 11}

In [ ]:
def get_key(val):
    for key, value in Classes.items():
        if val == value:
            return key

In [ ]:
def get_preds_to_labels(preds):
    pred_lists = []
    for pred in preds:
        pred_list = []
        health = (pred>=0.4)
        
        # get healthy
        if health.sum()==0:
            label = 'healthy'
            pred_list.append(label)
            
        elif pred[2]>=0.5:
            label = 'healthy'
            pred_list.append(label)
            
        # get eles label
        else:
            for j, sub in enumerate(pred):
                if sub>=0.28:
                    label = get_key(j)
                    pred_list.append(label)
                               
        pred_lists.append(' '.join(pred_list))
    return pred_lists

In [ ]:
preds_list = get_preds_to_labels(preds)

In [ ]:
submission['labels'] = preds_list
submission.head()

# SUBMISSION OF FILE

In [ ]:
submission.to_csv('submission.csv', index=False)